# **INF161 - Bike Traffic Prediction Project**
### *Ole Kristian Westby | owe009@uib.no | H23*

This project uses data from Statens vegvesen and Geofysisk institutt. The goal is to create a model that can predict the volume of bikers at a given time over Nygårdsbroen. I'll need to prepare the data so I'm left with the data I deem valuable to perform this task. That's what this Jupyter notebook is for. I'll also be explaining my steps throughout the book. At the end, we'll have some juicy, ready data that we'll use to insert into /ready_data/ ready for the model to work on.

I recognize that throughout the years there has been some times where people might have used the bikes more/less frequently because of certain factors. I will keep a list that I will update continuously as I find them.
- Covid-19 likely kept more people home, especially in peak times. Less people using bicycles to get to work as they had work from home. Only interested in peak covid-19 times though. 
- 2017 UCI Road World Championships. I've checked the routes and don't see that any bikes passed Nygårdsbroen but I will look closer at the data later.
- 

#### **Let's start by importing some libraries.**

In [355]:
import numpy as np
import pandas as pd
import os

#### **We'll handle the traffic data first.**

In [356]:
dir_weather = "raw_data/weather_data/"

files = [f for f in os.listdir(dir_weather) if f.endswith('.csv')]

# Interesting columns
columns = ["Dato", "Tid", "Solskinstid", "Lufttemperatur", "Vindstyrke", "Vindkast"]

dfs = []
for file in files:

    file_path = os.path.join(dir_weather, file)

    df = pd.read_csv(file_path, usecols=columns)

    dfs.append(df)

merged_weather_df = pd.concat(dfs, ignore_index=True)
merged_weather_df.tail()

,Dato,Tid,Solskinstid,Lufttemperatur,Vindstyrke,Vindkast
709216,2023-06-30,23:10,0.0,13.7,2.3,3.6
709217,2023-06-30,23:20,0.0,13.6,1.9,3.3
709218,2023-06-30,23:30,0.0,13.6,1.7,3.0
709219,2023-06-30,23:40,0.0,13.6,1.9,3.3
709220,2023-06-30,23:50,0.0,13.5,1.9,3.0


#### **Now we've created one big dataframe containing all interesting weather data from 2010 to 2023. However, the traffic data only goes from 2015-2023, and so I want to clear the dataset for any weather data before that. The model is going to get slightly less accurate with less data, but I think there's enough data already with 2015-2023 to do this anyways. It makes it simpler as well. I won't remove the data from the raw_data folder because I still recognize it there, and I want to see the different predictions based on it being included or not, but for now I won't focus on it.**

In [357]:
merged_weather_df["Dato"] = pd.to_datetime(merged_weather_df["Dato"])

merged_weather_df = merged_weather_df[merged_weather_df["Dato"].dt.year >= 2015]
merged_weather_df = merged_weather_df.reset_index(drop=True)

merged_weather_df.head()

,Dato,Tid,Solskinstid,Lufttemperatur,Vindstyrke,Vindkast
0,2015-01-01,00:00,0.0,6.6,4.2,NaN
1,2015-01-01,00:10,0.0,6.6,4.0,NaN
2,2015-01-01,00:20,0.0,6.6,3.1,NaN
3,2015-01-01,00:30,0.0,6.6,3.7,NaN
4,2015-01-01,00:40,0.0,6.7,2.9,NaN


#### **As we see in the beginning of the merged dataframe, we see some data missing in 2015-01-01 for Vindkast. I want clean, full data and who knows how many rows are missing data in one or more columns. Let's find out.**

In [358]:
rows_missing_data = merged_weather_df[merged_weather_df.isna().any(axis=1)].shape[0]
print(rows_missing_data)

3786


#### **As we can see, there are 3786 rows that are missing some data. 3786 is only 0,84% of the merged dataframe. I think we can afford to get rid of that.**

In [359]:
merged_weather_df.dropna(inplace=True)

merged_weather_df = merged_weather_df.reset_index(drop=True)

merged_weather_df.tail()

,Dato,Tid,Solskinstid,Lufttemperatur,Vindstyrke,Vindkast
442963,2023-06-30,23:10,0.0,13.7,2.3,3.6
442964,2023-06-30,23:20,0.0,13.6,1.9,3.3
442965,2023-06-30,23:30,0.0,13.6,1.7,3.0
442966,2023-06-30,23:40,0.0,13.6,1.9,3.3
442967,2023-06-30,23:50,0.0,13.5,1.9,3.0


#### **The next thing I want to do is combine the columns "Dato" and "Tid" to get a single datetime column. This will be useful later.**

In [360]:
merged_weather_df["Dato"] = merged_weather_df["Dato"].astype(str)
merged_weather_df["Tid"] = merged_weather_df["Tid"].astype(str)

merged_weather_df["Datotid"] = merged_weather_df["Dato"] + " " + merged_weather_df["Tid"]

merged_weather_df["Datotid"] = pd.to_datetime(merged_weather_df["Datotid"])

merged_weather_df.drop(["Dato", "Tid"], axis=1, inplace=True)

print(merged_weather_df)

        Solskinstid  Lufttemperatur  Vindstyrke  Vindkast             Datotid
0               0.0             4.4         1.3       3.6 2015-01-08 15:30:00
1               0.0             4.7         1.6       2.7 2015-01-08 15:40:00
2               0.0             4.5         1.9       2.7 2015-01-08 15:50:00
3               0.0             4.2         3.2       5.4 2015-01-08 16:00:00
4               0.0             4.5         2.8       4.5 2015-01-08 16:10:00
...             ...             ...         ...       ...                 ...
442963          0.0            13.7         2.3       3.6 2023-06-30 23:10:00
442964          0.0            13.6         1.9       3.3 2023-06-30 23:20:00
442965          0.0            13.6         1.7       3.0 2023-06-30 23:30:00
442966          0.0            13.6         1.9       3.3 2023-06-30 23:40:00
442967          0.0            13.5         1.9       3.0 2023-06-30 23:50:00

[442968 rows x 5 columns]


#### **The next we'll do is look at the traffic data**

In [361]:
dir_traffic = "raw_data/traffic_data/trafikkdata.csv"
traffic_df = pd.read_csv(dir_traffic, delimiter=";")

traffic_df.tail()

C:\Users\owe05\AppData\Local\Temp\ipykernel_9276\192785748.py:2: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  traffic_df = pd.read_csv(dir_traffic, delimiter=";")


,Trafikkregistreringspunkt,Navn,Vegreferanse,Fra,Til,Dato,Fra tidspunkt,Til tidspunkt,Felt,Trafikkmengde,"Dekningsgrad (%)|Antall timer total|Antall timer inkludert|Antall timer ugyldig|Ikke gyldig lengde|Lengdekvalitetsgrad (%)|< 5,6m|>= 5,6m|5,6m - 7,6m|7,6m - 12,5m|12,5m - 16,0m|>= 16,0m|16,0m - 24,0m|>= 24,0m"
348635,17510B2483952,Gamle Nygårdsbru sykkel,KV256 S2D1 m75,2023-07-01T23:00+02:00,2023-07-02T00:00+02:00,2023-07-01,23:00,00:00,1,NaN,|||||||||||||
348636,17510B2483952,Gamle Nygårdsbru sykkel,KV256 S2D1 m75,2023-07-01T23:00+02:00,2023-07-02T00:00+02:00,2023-07-01,23:00,00:00,2,NaN,|||||||||||||
348637,17510B2483952,Gamle Nygårdsbru sykkel,KV256 S2D1 m75,2023-07-01T23:00+02:00,2023-07-02T00:00+02:00,2023-07-01,23:00,00:00,Totalt i retning Danmarksplass,NaN,|||||||||||||
348638,17510B2483952,Gamle Nygårdsbru sykkel,KV256 S2D1 m75,2023-07-01T23:00+02:00,2023-07-02T00:00+02:00,2023-07-01,23:00,00:00,Totalt i retning Florida,NaN,|||||||||||||
348639,17510B2483952,Gamle Nygårdsbru sykkel,KV256 S2D1 m75,2023-07-01T23:00+02:00,2023-07-02T00:00+02:00,2023-07-01,23:00,00:00,Totalt,NaN,|||||||||||||


#### **It's clear there's A LOT of information we don't need/want for the project. I am happy to see the amount of rows aren't that far away from my weather dataframe.**

In [362]:
traffic_df = (
    traffic_df[traffic_df["Felt"] == "Totalt"]
    .loc[:, ["Dato", "Fra tidspunkt", "Trafikkmengde"]]
    .assign(Datotid=lambda x: pd.to_datetime(x["Dato"] + " " + x["Fra tidspunkt"]))
    .drop(columns=["Dato", "Fra tidspunkt"])
)
traffic_df.dropna(inplace=True)
traffic_df = traffic_df.reset_index(drop=True)
traffic_df.set_index("Datotid", inplace=True)

print(traffic_df)

                    Trafikkmengde
Datotid                          
2015-07-16 15:00:00             -
2015-07-16 16:00:00           101
2015-07-16 17:00:00            79
2015-07-16 18:00:00            56
2015-07-16 19:00:00            45
...                           ...
2022-12-31 19:00:00           0.0
2022-12-31 20:00:00           0.0
2022-12-31 21:00:00           3.0
2022-12-31 22:00:00           5.0
2022-12-31 23:00:00           1.0

[65361 rows x 1 columns]


#### **We now have two dataframes that are sorted for only information I need. However, every row in the weather dataframe is 10 minutes, while the traffic dataframe is an hour. I will have to reduce the amount of rows in weather so it shows hourly instead.**

In [363]:
merged_weather_df.set_index("Datotid", inplace=True)

# Perfect opportunity to rename the variable.
weather_df = merged_weather_df.resample("H").mean()

ready_df = traffic_df.join(weather_df, how="inner")
ready_df.reset_index(inplace=True)

ready_df.tail(10)

,Datotid,Trafikkmengde,Solskinstid,Lufttemperatur,Vindstyrke,Vindkast
65351,2022-12-31 14:00:00,11.0,0.0,3.566667,2.566667,4.55
65352,2022-12-31 15:00:00,13.0,0.0,3.133333,1.533333,3.20
65353,2022-12-31 16:00:00,6.0,0.0,2.533333,0.850000,1.55
65354,2022-12-31 17:00:00,2.0,0.0,1.883333,0.983333,1.75
65355,2022-12-31 18:00:00,8.0,0.0,2.016667,1.400000,2.75
65356,2022-12-31 19:00:00,0.0,0.0,1.766667,1.283333,2.45
65357,2022-12-31 20:00:00,0.0,0.0,0.666667,1.700000,3.15
65358,2022-12-31 21:00:00,3.0,0.0,0.483333,0.833333,1.60
65359,2022-12-31 22:00:00,5.0,0.0,0.516667,2.166667,4.20
65360,2022-12-31 23:00:00,1.0,0.0,0.316667,0.466667,1.10


#### **There's the final product. Ish.. we still need to train the model on this data. I was originally worried we wouldn't have too many rows available but it seems with 65361 rows, I am sure that's enough to get some good predictions!**